# pykan Results

Some preliminaries

In [ ]:
import torch
from torch import autograd
from tqdm import tqdm
from kan import KAN as pykan

import time
from scipy.stats.qmc import Sobol
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
import scipy
device = torch.device("cuda")

In [ ]:
# Used for first-order derivatives
def batch_jacobian(func, x, create_graph=False):
    def _func_sum(x):
        return func(x).sum(dim=0)
    return autograd.functional.jacobian(_func_sum, x, create_graph=create_graph).permute(1, 0, 2)

# Define Sobol Sampling
def sobol_colloc(X0, X1, N, seed=42):
    """
    N - number of colloc points
    
    """
    dims = X0.shape[0]
    sobol_sampler = Sobol(dims, scramble=True, seed=seed)
    points = sobol_sampler.random_base2(int(np.log2(N)))
    points = X0 + points * (X1 - X0)
    
    return points

## Diffusion Equation

### Collocation Points

In [ ]:
N = 2**12
collocs = np.array(sobol_colloc(np.array([0,-1]), np.array([1,1]), N)) # (4096, 2)

# Generate Collocation points for BCs
N = 2**6

BC1_colloc = np.array(sobol_colloc(np.array([0,-1]), np.array([0,1]), N)) # (64, 2)
BC1_data =  np.sin(np.pi*BC1_colloc[:,1]).reshape(-1,1) # (64, 1)

BC2_colloc = np.array(sobol_colloc(np.array([0,-1]), np.array([1,-1]), N)) # (64, 2)
BC2_data = np.zeros(BC2_colloc.shape[0]).reshape(-1,1) # (64, 1)

BC3_colloc = np.array(sobol_colloc(np.array([0,1]), np.array([1,1]), N)) # (64, 2)
BC3_data = np.zeros(BC3_colloc.shape[0]).reshape(-1,1) # (64, 1)

# Get collocation points from jaxkan application
tcollocs = torch.from_numpy(collocs).to(device)
tcollocs.requires_grad = True

# Get boundary condition points and data from jaxkan application
tbc_collocs = [torch.from_numpy(BC1_colloc).to(device),
              torch.from_numpy(BC2_colloc).to(device),
              torch.from_numpy(BC3_colloc).to(device)]

tbc_data = [torch.from_numpy(BC1_data).to(device),
           torch.from_numpy(BC2_data).to(device),
           torch.from_numpy(BC3_data).to(device)]

### Loss Function & training

In [ ]:
grid_vals = [3] # Grid sizes to use
init_lr = 0.001
num_epochs = 50000

t_losses = []

tstart_time = time.time()
for grid in grid_vals:
    if grid == grid_vals[0]:
        model = pykan(width=[2,6,6,1], grid=3, k=3, grid_eps=0.05, symbolic_enabled=False, device=device)
        model.update_grid_from_samples(tcollocs)
        model = model.to(device)
    else:
        model = pykan(width=[2,6,6,1], grid=grid, k=3, grid_eps=0.05, symbolic_enabled=False, device=device).initialize_from_another_model(model, collocs)
        model = model.to(device)

    # Define the training loop
    def train():
    
        optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

        # Parameters
        D = torch.tensor(0.15, device=device, requires_grad=False)
        w_pde = torch.tensor(1.0, device=device, requires_grad=False)
        w_bcs = [torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False)]

        def closure():
            optimizer.zero_grad()
            
            # The kan output has shape (batch, kan_output)
            u = model(tcollocs)

            # Define u_t and u_x functions
            u_t_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,0].unsqueeze(1)
            u_x_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Define u_xx function
            u_xx_fun = lambda x: batch_jacobian(u_x_fun, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Get u_t and u_x with shapes (batch,1)
            u_t = u_t_fun(tcollocs)
            #u_x = u_x_fun(tcollocs)
            # Get u_xx with shape (batch,1)
            u_xx = u_xx_fun(tcollocs)

            # PDE Loss
            source=torch.exp(-tcollocs[:,[0]])*(-torch.sin(torch.pi*tcollocs[:,[1]])+torch.pi**2*torch.sin(torch.pi*tcollocs[:,[1]]))
            pde_loss = w_pde*torch.mean((u_t - u_xx-source)**2)
            
            # Boundary Conditions Loss
            bc_loss = 0.0
            for idx, _ in enumerate(tbc_collocs):
                # Get model's prediction
                pred = model(tbc_collocs[idx])
                # Retrieve true values from boundary conditions
                true = tbc_data[idx]
                # MSE Loss
                bc_loss += w_bcs[idx]*torch.mean((pred-true)**2)

            total_loss = pde_loss + bc_loss
            total_loss.backward()
            return total_loss
            
        for step in range(num_epochs):
    
            optimizer.step(closure)

            # Get Loss
            current_loss = closure().item()
            
            t_losses.append(current_loss)
    
    train()

tend_time = time.time()
telapsed = tend_time - tstart_time
print(f"Total Time: {telapsed} s")
print(f"Average time per iteration: {telapsed/num_epochs:.2f} s")

### Results

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(np.array(t_losses), label='Train Loss', marker='o', color='blue', markersize=1)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.yscale('log')  # Set y-axis to logarithmic scale

plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5) 

plt.show()

In [ ]:
N_t, N_x = 100, 256

tt = torch.linspace(0, 1.0, N_t, device='cpu', requires_grad=False)
tx = torch.linspace(0.0, 1.0, N_x, device='cpu', requires_grad=False)
tT, tX = torch.meshgrid(tt, tx, indexing='ij')
tcoords = torch.stack([tT.flatten(), tX.flatten()], dim=1).double().to(device)

tresplot = model(tcoords)[:,0].cpu().detach().reshape(N_t, N_x)

plt.figure(figsize=(10, 5))
plt.pcolormesh(tT, tX, tresplot, shading='auto', cmap='Spectral_r')
plt.colorbar()

plt.title('Solution of Diffusion Equation')
plt.xlabel('t')

plt.ylabel('x')

plt.tight_layout()
plt.show()

In [ ]:
torch.save(model.state_dict(), 'pykan models/eq2-pykan')
np.savez('../Plots/data/eq2-pykan.npz', arr_0=tresplot)

## Helmholtz Equation

### Collocation Points

In [ ]:
# Generate Collocation points for PDE
N = 2**12
collocs = np.array(sobol_colloc(np.array([-1,-1]), np.array([1,1]), N)) # (4096, 2)

# Generate Collocation points for BCs
N = 2**6

BC1_colloc = np.array(sobol_colloc(np.array([-1,-1]), np.array([-1,1]), N)) # (64, 2)
BC1_data =  np.zeros(BC1_colloc.shape[0]).reshape(-1,1) # (64, 1)

BC2_colloc = np.array(sobol_colloc(np.array([-1,-1]), np.array([1,-1]), N)) # (64, 2)
BC2_data = np.zeros(BC2_colloc.shape[0]).reshape(-1,1) # (64, 1)

BC3_colloc = np.array(sobol_colloc(np.array([-1,1]), np.array([1,1]), N)) # (64, 2)
BC3_data = np.zeros(BC3_colloc.shape[0]).reshape(-1,1) # (64, 1)

BC4_colloc = np.array(sobol_colloc(np.array([1,-1]), np.array([1,1]), N)) # (64, 2)
BC4_data = np.zeros(BC4_colloc.shape[0]).reshape(-1,1) # (64, 1)

# Get collocation points from jaxkan application
tcollocs = torch.from_numpy(collocs).to(device)
tcollocs.requires_grad = True

# Get boundary condition points and data from jaxkan application
tbc_collocs = [torch.from_numpy(BC1_colloc).to(device),
              torch.from_numpy(BC2_colloc).to(device),
              torch.from_numpy(BC3_colloc).to(device),
              torch.from_numpy(BC4_colloc).to(device)]

tbc_data = [torch.from_numpy(BC1_data).to(device),
           torch.from_numpy(BC2_data).to(device),
           torch.from_numpy(BC3_data).to(device),
           torch.from_numpy(BC4_data).to(device)]

### Loss Function & Training

In [ ]:
grid_vals = [3] # Grid sizes to use
init_lr = 0.001
num_epochs = 50000

t_losses = []

tstart_time = time.time()
for grid in grid_vals:
    if grid == grid_vals[0]:
        model = pykan(width=[2,6,6,1], grid=3, k=3, grid_eps=0.05, symbolic_enabled=False, device=device)
        model.update_grid_from_samples(tcollocs)
        model = model.to(device)
    else:
        model = pykan(width=[2,6,6,1], grid=grid, k=3, grid_eps=0.05, symbolic_enabled=False, device=device).initialize_from_another_model(model, collocs)
        model = model.to(device)

    # Define the training loop
    def train():
    
        optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

        # Parameters
        a1 = torch.tensor(1.0, device=device, requires_grad=False)
        a2 = torch.tensor(4.0, device=device, requires_grad=False)
        k = torch.tensor(1.0, device=device, requires_grad=False)
        w_pde = torch.tensor(1.0, device=device, requires_grad=False)
        w_bcs = [torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False)]

        def closure():
            optimizer.zero_grad()
            
            # The kan output has shape (batch, kan_output)
            u = model(tcollocs)

            # Define u_t and u_x functions
            u_x_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,0].unsqueeze(1)
            u_y_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Define u_xx function
            u_xx_fun = lambda x: batch_jacobian(u_x_fun, x, create_graph=True)[:,0,0].unsqueeze(1)
            u_yy_fun = lambda x: batch_jacobian(u_y_fun, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Get u_t and u_x with shapes (batch,1)
            u_xx = u_xx_fun(tcollocs)
            #u_x = u_x_fun(tcollocs)
            # Get u_xx with shape (batch,1)
            u_yy = u_yy_fun(tcollocs)

            # PDE Loss
            sina1sina2=torch.sin(a1*torch.pi*tcollocs[:,[0]])*torch.sin(a2*torch.pi*tcollocs[:,[1]])
            q= -(a1*torch.pi)**2*sina1sina2 -(a2*torch.pi)**2*sina1sina2 +k*sina1sina2
            
            pde_loss = w_pde*torch.mean((u_xx + u_yy +k**2 *u - q)**2)
            
            # Boundary Conditions Loss
            bc_loss = 0.0
            for idx, _ in enumerate(tbc_collocs):
                # Get model's prediction
                pred = model(tbc_collocs[idx])
                # Retrieve true values from boundary conditions
                true = tbc_data[idx]
                # MSE Loss
                bc_loss += w_bcs[idx]*torch.mean((pred-true)**2)

            total_loss = pde_loss + bc_loss
            total_loss.backward()
            return total_loss
            
        for step in range(num_epochs):
    
            optimizer.step(closure)

            # Get Loss
            current_loss = closure().item()
            
            t_losses.append(current_loss)
    
    train()

tend_time = time.time()
telapsed = tend_time - tstart_time
print(f"Total Time: {telapsed} s")
print(f"Average time per iteration: {telapsed/num_epochs:.2f} s")

### Results

In [ ]:
N_t, N_x = 100, 256

tt = torch.linspace(0, 1.0, N_t, device='cpu', requires_grad=False)
tx = torch.linspace(0.0, 1.0, N_x, device='cpu', requires_grad=False)
tT, tX = torch.meshgrid(tt, tx, indexing='ij')
tcoords = torch.stack([tT.flatten(), tX.flatten()], dim=1).double().to(device)

tresplot = model(tcoords)[:,0].cpu().detach().reshape(N_t, N_x)
plt.figure(figsize=(10, 5))
plt.pcolormesh(tT, tX, tresplot, shading='auto', cmap='Spectral_r')
plt.colorbar()

plt.title('Solution of helmholtz Equation')
plt.xlabel('t')

plt.ylabel('x')

plt.tight_layout()
plt.show()

In [ ]:
a1=1.0
a2=4.0
k=1.0

def u_exact(t,x): 
    return np.sin(a1*np.pi*t)*np.sin(a2*np.pi*x)

In [ ]:
residuals = tresplot - u_exact(tT, tX)

plt.figure(figsize=(10, 5))
plt.pcolormesh(tT, tX, residuals, shading='auto', cmap='seismic')#,vmin=-100,vmax=100)
plt.colorbar()

plt.title('residuals of Diffusion Equation')
plt.xlabel('t')

plt.ylabel('x')

plt.tight_layout()
plt.show()

In [ ]:
torch.save(model.state_dict(), 'pykan models/eq2-pykan')
np.savez('../Plots/data/eq2-pykan.npz', arr_0=tresplot)

# Burgers Equation

## Collocation Points

In [ ]:
# Generate Collocation points for PDE
N = 2**12
collocs = np.array(sobol_colloc(np.array([-1,-1]), np.array([1,1]), N)) # (4096, 2)

# Generate Collocation points for BCs
N = 2**6

BC1_colloc = np.array(sobol_colloc(np.array([0,-1]), np.array([0,1]), N)) # (64, 2)
BC1_data = - np.sin(np.pi*BC1_colloc[:,1]).reshape(-1,1) # (64, 1)

BC2_colloc = np.array(sobol_colloc(np.array([0,-1]), np.array([1,-1]), N)) # (64, 2)
BC2_data = np.zeros(BC2_colloc.shape[0]).reshape(-1,1) # (64, 1)

BC3_colloc = np.array(sobol_colloc(np.array([0,1]), np.array([1,1]), N)) # (64, 2)
BC3_data = np.zeros(BC3_colloc.shape[0]).reshape(-1,1) # (64, 1)

# Get collocation points from jaxkan application
tcollocs = torch.from_numpy(collocs).to(device)
tcollocs.requires_grad = True

# Get boundary condition points and data from jaxkan application
tbc_collocs = [torch.from_numpy(BC1_colloc).to(device),
              torch.from_numpy(BC2_colloc).to(device),
              torch.from_numpy(BC3_colloc).to(device)]

tbc_data = [torch.from_numpy(BC1_data).to(device),
           torch.from_numpy(BC2_data).to(device),
           torch.from_numpy(BC3_data).to(device)]

## Loss function & training

In [ ]:
grid_vals = [3] # Grid sizes to use
init_lr = 0.001
num_epochs = 50000

t_losses = []

tstart_time = time.time()
for grid in grid_vals:
    if grid == grid_vals[0]:
        model = pykan(width=[2,6,6,1], grid=3, k=3, grid_eps=0.05, symbolic_enabled=False, device=device)
        model.update_grid_from_samples(tcollocs)
        model = model.to(device)
    else:
        model = pykan(width=[2,6,6,1], grid=grid, k=3, grid_eps=0.05, symbolic_enabled=False, device=device).initialize_from_another_model(model, collocs)
        model = model.to(device)

    # Define the training loop
    def train():
    
        optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

        # Parameters
        v = torch.tensor(0.01/torch.pi, device=device, requires_grad=False)
        w_pde = torch.tensor(1.0, device=device, requires_grad=False)
        w_bcs = [torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False)]

        def closure():
            optimizer.zero_grad()
            
            # The kan output has shape (batch, kan_output)
            u = model(tcollocs)

            # Define u_t and u_x functions
            u_t_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,0].unsqueeze(1)
            u_x_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Define u_xx function
            u_xx_fun = lambda x: batch_jacobian(u_x_fun, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Get u_t and u_x with shapes (batch,1)
            u_t = u_t_fun(tcollocs)
            u_x = u_x_fun(tcollocs)
            # Get u_xx with shape (batch,1)
            u_xx = u_xx_fun(tcollocs)

            # PDE Loss
            pde_loss = w_pde*torch.mean((u_t + u*u_x - v*u_xx)**2)
            
            # Boundary Conditions Loss
            bc_loss = 0.0
            for idx, _ in enumerate(tbc_collocs):
                # Get model's prediction
                pred = model(tbc_collocs[idx])
                # Retrieve true values from boundary conditions
                true = tbc_data[idx]
                # MSE Loss
                bc_loss += w_bcs[idx]*torch.mean((pred-true)**2)

            total_loss = pde_loss + bc_loss
            total_loss.backward()
            return total_loss
            
        for step in range(num_epochs):
    
            optimizer.step(closure)

            # Get Loss
            current_loss = closure().item()
            
            t_losses.append(current_loss)
    
    train()

tend_time = time.time()
telapsed = tend_time - tstart_time
print(f"Total Time: {telapsed} s")
print(f"Average time per iteration: {telapsed/num_epochs:.2f} s")

## Results

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(np.array(t_losses), label='Train Loss', marker='o', color='blue', markersize=1)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.yscale('log')  # Set y-axis to logarithmic scale

plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5) 

plt.show()

In [ ]:
N_t, N_x = 100, 256

tt = torch.linspace(0, 1.0, N_t, device='cpu', requires_grad=False)
tx = torch.linspace(-1.0, 1.0, N_x, device='cpu', requires_grad=False)
tT, tX = torch.meshgrid(tt, tx, indexing='ij')
tcoords = torch.stack([tT.flatten(), tX.flatten()], dim=1).double().to(device)

tresplot = model(tcoords)[:,0].cpu().detach().reshape(N_t, N_x)
plt.figure(figsize=(10, 5))
plt.pcolormesh(tT, tX, tresplot, shading='auto', cmap='Spectral_r')
plt.colorbar()

plt.title('Solution of Burgers Equation')
plt.xlabel('t')

plt.ylabel('x')

plt.tight_layout()
plt.show()

In [ ]:
Bsol=np.load('../External Data/Burgers.npz')
Bsol_interp=RegularGridInterpolator((Bsol['t'][:,0], Bsol['x'][:,0]) ,Bsol['usol'].T,method='linear',bounds_error=False,fill_value=np.nan)

def u_exact(t,x): return Bsol_interp((t, t))

residuals = tresplot - u_exact(tT, tX)
plt.figure(figsize=(10, 5))
plt.pcolormesh(tT, tX, residuals, shading='auto', cmap='seismic')#,vmin=-100,vmax=100)
plt.colorbar()

plt.title('Solution of Burgers Equation')
plt.xlabel('t')

plt.ylabel('x')

plt.tight_layout()
plt.show()

In [ ]:
torch.save(model.state_dict(), 'pykan models/eq3-pykan')
np.savez('../Plots/data/eq3-pykan.npz', arr_0=tresplot)

# Allen Cahn Equation

## Collocation Points

In [ ]:
# Generate Collocation points for PDE
N = 2**12
collocs = np.array(sobol_colloc(np.array([0,-1]), np.array([1,1]), N)) # (4096, 2)

# Generate Collocation points for BCs
N = 2**6

BC1_colloc = np.array(sobol_colloc(np.array([0,-1]), np.array([0,1]), N)) # (64, 2)
BC1_data =  ((BC1_colloc[:,1]**2)*np.cos(np.pi*BC1_colloc[:,1])).reshape(-1,1)

BC2_colloc = np.array(sobol_colloc(np.array([0,-1]), np.array([1,-1]), N)) # (64, 2)
BC2_data = -np.ones(BC2_colloc.shape[0]).reshape(-1,1) # (64, 1)

BC3_colloc = np.array(sobol_colloc(np.array([0,1]), np.array([1,1]), N)) # (64, 2)
BC3_data = -np.ones(BC3_colloc.shape[0]).reshape(-1,1) # (64, 1)

# Get collocation points from jaxkan application
tcollocs = torch.from_numpy(collocs).to(device)
tcollocs.requires_grad = True

# Get boundary condition points and data from jaxkan application
tbc_collocs = [torch.from_numpy(BC1_colloc).to(device),
              torch.from_numpy(BC2_colloc).to(device),
              torch.from_numpy(BC3_colloc).to(device)]

tbc_data = [torch.from_numpy(BC1_data).to(device),
           torch.from_numpy(BC2_data).to(device),
           torch.from_numpy(BC3_data).to(device)]

## Loss Function & training

In [ ]:
grid_vals = [3] # Grid sizes to use
init_lr = 0.001
num_epochs = 50000

t_losses = []

tstart_time = time.time()
for grid in grid_vals:
    if grid == grid_vals[0]:
        model = pykan(width=[2,6,6,1], grid=3, k=3, grid_eps=0.05, symbolic_enabled=False, device=device)
        model.update_grid_from_samples(tcollocs)
        model = model.to(device)
    else:
        model = pykan(width=[2,6,6,1], grid=grid, k=3, grid_eps=0.05, symbolic_enabled=False, device=device).initialize_from_another_model(model, collocs)
        model = model.to(device)

    # Define the training loop
    def train():
    
        optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)

        # Parameters
        D = torch.tensor(0.001, device=device, requires_grad=False)
        c = torch.tensor(5.0, device=device, requires_grad=False)
        w_pde = torch.tensor(1.0, device=device, requires_grad=False)
        w_bcs = [torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False),
                 torch.tensor(1.0, device=device, requires_grad=False)]

        def closure():
            optimizer.zero_grad()
            
            # The kan output has shape (batch, kan_output)
            u = model(tcollocs)

            # Define u_t and u_x functions
            u_t_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,0].unsqueeze(1)
            u_x_fun = lambda x: batch_jacobian(model, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Define u_xx function
            u_xx_fun = lambda x: batch_jacobian(u_x_fun, x, create_graph=True)[:,0,1].unsqueeze(1)

            # Get u_t and u_x with shapes (batch,1)
            u_t = u_t_fun(tcollocs)
            #u_x = u_x_fun(tcollocs)
            # Get u_xx with shape (batch,1)
            u_xx = u_xx_fun(tcollocs)

            # PDE Loss
            pde_loss = w_pde*torch.mean((u_t - D*u_xx-c*(u-u**3))**2)
            
            # Boundary Conditions Loss
            bc_loss = 0.0
            for idx, _ in enumerate(tbc_collocs):
                # Get model's prediction
                pred = model(tbc_collocs[idx])
                # Retrieve true values from boundary conditions
                true = tbc_data[idx]
                # MSE Loss
                bc_loss += w_bcs[idx]*torch.mean((pred-true)**2)

            total_loss = pde_loss + bc_loss
            total_loss.backward()
            return total_loss
            
        for step in range(num_epochs):
    
            optimizer.step(closure)

            # Get Loss
            current_loss = closure().item()
            
            t_losses.append(current_loss)
    
    train()

tend_time = time.time()
telapsed = tend_time - tstart_time
print(f"Total Time: {telapsed} s")
print(f"Average time per iteration: {telapsed/num_epochs:.2f} s")

## Results

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(np.array(t_losses), label='Train Loss', marker='o', color='blue', markersize=1)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.yscale('log')  # Set y-axis to logarithmic scale

plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5) 

plt.show()

In [ ]:
N_t, N_x = 100, 256

tt = torch.linspace(0, 1.0, N_t, device='cpu', requires_grad=False)
tx = torch.linspace(-1.0, 1.0, N_x, device='cpu', requires_grad=False)
tT, tX = torch.meshgrid(tt, tx, indexing='ij')
tcoords = torch.stack([tT.flatten(), tX.flatten()], dim=1).double().to(device)

tresplot = model(tcoords)[:,0].cpu().detach().reshape(N_t, N_x)
plt.figure(figsize=(10, 5))
plt.pcolormesh(tT, tX, tresplot, shading='auto', cmap='Spectral_r')
plt.colorbar()

plt.title('Solution of Allen Cahn Equation')
plt.xlabel('t')

plt.ylabel('x')

plt.tight_layout()
plt.show()

In [ ]:
# Draw reference values
ref = scipy.io.loadmat('../External Data/usol_D_0.001_k_5.mat')

sol_interp=RegularGridInterpolator((ref['t'][0], ref['x'][0]) ,ref['u'],method='linear',bounds_error=False,fill_value=np.nan)

def u_exact(t,x): return sol_interp((t, x))

In [ ]:
residuals = tresplot - u_exact(tT, tX)
plt.figure(figsize=(10, 5))
plt.pcolormesh(tT, tX, residuals, shading='auto', cmap='seismic')#,vmin=-100,vmax=100)
plt.colorbar()

plt.title('Solution of Allen Cahn Equation')
plt.xlabel('t')

plt.ylabel('x')

plt.tight_layout()
plt.show()

In [ ]:
torch.save(model.state_dict(), 'pykan models/eq4-pykan')
np.savez('../Plots/data/eq4-pykan.npz', arr_0=tresplot)